# Imports

In [1]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys, os
# this adds the src folder in the sys path, where the metric_utils.py file is
# not needed if this notebook is in the same folder, but uncomment to access from the data subfolders
sys.path.append( '..' )
from metric_utils import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Result path

In [5]:
dataset_name = 'ucberkeley'
model_name = 'bert-base-uncased'
run_name = "run 1"

# change to ../../results/{dataset_name} when using inside one of the data subfolders
run_folder = f'../../results/{dataset_name}/{run_name}'
test_csv_filepath = os.path.join(run_folder, 'test.csv')

model_folder = os.path.join(run_folder, model_name) # for this particular model
normal_folder = os.path.join(model_folder, 'normal')
dp_folder = os.path.join(model_folder, 'epsilon 3.0')

result_filepath = os.path.join(normal_folder, 'results.csv')
dp_result_filepath = os.path.join(dp_folder, 'results.csv')

# Dataset
The dataset can be preprocessed from the original dataset to be used here. To simplify things, I saved the preprocessed datasets during the tokenizing process as csv files and then downloaded them in the corresponding dataset folder of [`result`](../results/) directory.

You can recreate the processed datasets using the tokenize notebooks for that particular dataset. That would give you train, test and validation csv files as well as the tokenized data.

In [27]:
result = pd.read_csv(result_filepath)
dp_result = pd.read_csv(dp_result_filepath)
test_df = pd.read_csv(test_csv_filepath)

In [7]:
test_df.head(2)

,comment_id,text,target_race_asian,target_race_black,target_race_latinx,target_race_middle_eastern,target_race_native_american,target_race_pacific_islander,target_race_white,target_race_other,target_race,target_religion_atheist,target_religion_buddhist,target_religion_christian,target_religion_hindu,target_religion_jewish,target_religion_mormon,target_religion_muslim,target_religion_other,target_religion,target_origin_immigrant,target_origin_migrant_worker,target_origin_specific_country,target_origin_undocumented,target_origin_other,target_origin,target_gender_men,target_gender_non_binary,target_gender_transgender_men,target_gender_transgender_unspecified,target_gender_transgender_women,target_gender_women,target_gender_other,target_gender,target_sexuality_bisexual,target_sexuality_gay,target_sexuality_lesbian,target_sexuality_straight,target_sexuality_other,target_sexuality,target_age_children,target_age_teenagers,target_age_young_adults,target_age_middle_aged,target_age_seniors,target_age_other,target_age,target_disability_physical,target_disability_cognitive,target_disability_neurological,target_disability_visually_impaired,target_disability_hearing_impaired,target_disability_unspecific,target_disability_other,target_disability,labels
0,41012,I don't want to see America treated like a garbage can. Luv from Canada.,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0
1,31056,@mike_pence @realDonaldTrump You're a Christian of convenience. You're a fake,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,0


## Split test result

In [8]:
# only calculate test result
result = result[result['split']=='test']
dp_result = dp_result[dp_result['split']=='test']

# drop split column
result.drop(columns=['split'], inplace=True)
dp_result.drop(columns=['split'], inplace=True)

## Prepare test data

In [28]:
# result has id column which is the same as the text ids from raw dataset
raw_id_column = 'comment_id'
test_df.rename({raw_id_column: id_column}, axis=1, inplace=True)

## Merge

In [10]:
extra_columns = [col for col in test_df.columns if col in result.columns and col!=id_column]

result = result.merge(test_df.drop(columns=extra_columns), on=id_column, how='inner').reset_index(drop=True)
dp_result = dp_result.merge(test_df.drop(columns=extra_columns), on=id_column, how='inner').reset_index(drop=True)

In [12]:
result.head(1)

,id,labels,probs,text,target_race_asian,target_race_black,target_race_latinx,target_race_middle_eastern,target_race_native_american,target_race_pacific_islander,target_race_white,target_race_other,target_race,target_religion_atheist,target_religion_buddhist,target_religion_christian,target_religion_hindu,target_religion_jewish,target_religion_mormon,target_religion_muslim,target_religion_other,target_religion,target_origin_immigrant,target_origin_migrant_worker,target_origin_specific_country,target_origin_undocumented,target_origin_other,target_origin,target_gender_men,target_gender_non_binary,target_gender_transgender_men,target_gender_transgender_unspecified,target_gender_transgender_women,target_gender_women,target_gender_other,target_gender,target_sexuality_bisexual,target_sexuality_gay,target_sexuality_lesbian,target_sexuality_straight,target_sexuality_other,target_sexuality,target_age_children,target_age_teenagers,target_age_young_adults,target_age_middle_aged,target_age_seniors,target_age_other,target_age,target_disability_physical,target_disability_cognitive,target_disability_neurological,target_disability_visually_impaired,target_disability_hearing_impaired,target_disability_unspecific,target_disability_other,target_disability
0,41012,0,0.405073,I don't want to see America treated like a garbage can. Luv from Canada.,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


# Evaluation

## Convert probability to prediction

In [13]:
result[prediction_column] = result[probability_column] >=0.5
dp_result[prediction_column] = dp_result[probability_column] >=0.5

## Identity groups

In [21]:
group_map = {
    'gender': {
        'unprivileged':['target_gender_women'],
        'privileged':['target_gender_men']
    },
    'race': {
        'unprivileged':['target_race_black'],
        'privileged': ['target_race_white']
    }
}

identities = []
for group_key in group_map.keys():
    subgroup_map = group_map[group_key]
    for subgroup_key in subgroup_map.keys():
        identities.extend(subgroup_map[subgroup_key])

print(identities)

['target_gender_women', 'target_gender_men', 'target_race_black', 'target_race_white']


### Binarize identity and target columns

In [22]:
result = binarize(result, [target_column] + identities)
dp_result = binarize(dp_result, [target_column] + identities)

## Bias

In [23]:
bias_results = {
   'fairness_metrics': ['demographic parity', 'Equality of Opportunity (w.r.t y = 1)',
   'Equality of Opportunity (w.r.t y = 0)', 'Equality of Odds', 'unprotected-accuracy',
   'protected-accuracy', 'accuracy']
}

for group_key in group_map.keys():
   subgroup_map = group_map[group_key]
   privileged_group = subgroup_map['privileged']
   unprivileged_group = subgroup_map['unprivileged']

   bias_results[group_key] = calculate_bias(result, privileged_group, unprivileged_group)
   bias_results[group_key+'_DP'] = calculate_bias(dp_result, privileged_group, unprivileged_group)

bias_results = pd.DataFrame(bias_results) 
bias_results

,fairness_metrics,gender,gender_DP,race,race_DP
0,demographic parity,0.483063,0.708395,0.526616,0.576046
1,Equality of Opportunity (w.r.t y = 1),0.928221,0.993791,0.678739,0.639412
2,Equality of Opportunity (w.r.t y = 0),0.925489,0.981742,0.933954,0.932468
3,Equality of Odds,0.926855,0.987766,0.806347,0.785940
4,unprotected-accuracy,0.753078,0.729747,0.811206,0.771011
5,protected-accuracy,0.802651,0.774669,0.787072,0.766160
6,accuracy,0.777865,0.752208,0.799139,0.768585


In [12]:
bias_results.round(3).to_csv(os.path.join(dp_folder, 'bias.csv'), index=False)

## Overall metrics

In [24]:
overall_results = get_overall_results(group_map, result, dp_result)
print(overall_results)

,metrics,target_gender_men,target_gender_men_DP,target_gender_women,target_gender_women_DP,target_race_white,target_race_white_DP,target_race_black,target_race_black_DP,Total,Total_DP
0,size,679.000000,679.000000,1543.000000,1543.000000,526.000000,526.000000,821.000000,821.000000,5785.000000,5785.000000
1,auc,0.829878,0.784669,0.816385,0.771930,0.800283,0.752489,0.873806,0.850883,0.855806,0.812212
2,accuracy,0.802651,0.774669,0.753078,0.729747,0.787072,0.766160,0.811206,0.771011,0.808470,0.785134
3,f1_score,0.591463,0.474227,0.621648,0.489596,0.525424,0.381910,0.778255,0.704403,0.635286,0.515777
4,precision,0.642384,0.605263,0.623506,0.641026,0.620000,0.603175,0.779370,0.783217,0.673883,0.688866
5,recall,0.548023,0.389831,0.619802,0.396040,0.455882,0.279412,0.777143,0.640000,0.600872,0.412204
6,false positive rate,0.107570,0.089641,0.182081,0.107900,0.097436,0.064103,0.163482,0.131635,0.111749,0.071548
7,bnsp_auc,0.848152,0.826320,0.868274,0.827919,0.815554,0.780683,0.910230,0.882368,NaN,NaN
8,bpsn_auc,0.842582,0.776037,0.820976,0.770492,0.852754,0.796578,0.805964,0.762747,NaN,NaN


In [14]:
overall_results.round(3).to_csv(os.path.join(dp_folder, 'overall_results.csv'), index=False)

## Overall bias auc

In [25]:
overall_auc = overall_results[overall_results['metrics']=='auc']['Total'].values[0]

temp = overall_results.set_index('metrics').T
privileged, unprivileged = [], []
for group_key in group_map.keys():
    privileged.extend(group_map[group_key]['privileged'])
    unprivileged.extend(group_map[group_key]['unprivileged'])

privileged_bias_auc = get_final_metric(temp[temp.index.isin(privileged)], overall_auc)
unprivileged_bias_auc = get_final_metric(temp[temp.index.isin(unprivileged)], overall_auc)

privileged_bias_auc_dp = get_final_metric(
    temp[temp.index.isin([i+'_DP' for i in privileged])], 
    overall_auc
)

unprivileged_bias_auc_dp = get_final_metric(
    temp[temp.index.isin([i+'_DP' for i in unprivileged])], 
    overall_auc
)
print(privileged_bias_auc, unprivileged_bias_auc, privileged_bias_auc_dp, unprivileged_bias_auc_dp)

0.8371387851667849 0.8497560371852113 0.8027114661900272 0.820156122398146


# Count

The the `train.csv` file from `experiment/run` folders for this corresponding run. And manually copy it in the path run_folder points to.

In [29]:
train_df = pd.read_csv(os.path.join(run_folder, 'train.csv'))

count_dict = {
    'Identity':identities,
    '0 (train)':[],
    '1 (train)':[],
    '0 (test)':[],
    '1 (test)':[],
}
for identity in identities:
    train_neg, train_pos = train_df[train_df[identity]>=0.5][target_column].value_counts().to_numpy()
    test_neg, test_pos = test_df[test_df[identity]>=0.5][target_column].value_counts().to_numpy()
    count_dict['0 (train)'].append(train_neg)
    count_dict['1 (train)'].append(train_pos)
    count_dict['0 (test)'].append(test_neg)
    count_dict['1 (test)'].append(test_pos)

count_df = pd.DataFrame(count_dict)
print(count_df)

count_df.to_csv(os.path.join(run_folder, 'count.csv'), index=False)

              Identity  0 (train)  1 (train)  0 (test)  1 (test)
0  target_gender_women       4847       2304      1038       505
1    target_gender_men       2359        811       502       177
2    target_race_black       2136       1557       471       350
3    target_race_white       1689        634       390       136
